In [ ]:
from collections import namedtuple
import csv
from enum import Enum
from operator import attrgetter, itemgetter


ZERO_ADDR = '0x0000000000000000000000000000000000000000'
USDC_ADDR = '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'
DAI_ADDR = '0x6b175474e89094c44da98b954eedeac495271d0f'
USDT_ADDR = '0xdac17f958d2ee523a2206206994597c13d831ec7'
START_TIMESTAMP = 1597017600  # 10 Aug 2020 00:00 UTC
STOP_TIMESTAMP = 1607731200  # 12 Dec 2020 00:00 UTC
TOTAL_INCH_TOKENS = 1_500_000_000 * 10 ** 18
TOTAL_INCH_AIRDROP = TOTAL_INCH_TOKENS * 2 // 100


class Operation(Enum):
    ADDITION = 1
    REMOVAL = 2


LPTransfer = namedtuple('LPTransfer', ['timestamp', 'pool', 'addr_from', 'addr_to', 'amount'])
TokenTransfer = namedtuple('TokenTransfer', ['timestamp', 'pool', 'token', 'operation', 'amount'])
Price = namedtuple('Price', ['timestamp', 'price'])


def get_eth_transfers(pools):
    with open('data/bq-eth.csv') as csvfile:
        spamreader = csv.reader(csvfile)
        rows = [row for row in spamreader][1:]

    eth_transfers = dict()
    for row in rows:
        if row[2] in pools:
            assert row[3] not in pools
            pool = row[2]
            operation = Operation.REMOVAL
        elif row[3] in pools:
            pool = row[3]
            operation = Operation.ADDITION
        else:
            assert False
        if pool not in eth_transfers:
            eth_transfers[pool] = []
        eth_transfers[pool].append(TokenTransfer(int(row[0]), pool, ZERO_ADDR, operation, int(row[4])))
    
    print('Got ETH transfers for {} pools'.format(len(eth_transfers)))
    
    return eth_transfers


def get_token_transfers(pools):
    with open('data/bq-erc20.csv') as csvfile:
        spamreader = csv.reader(csvfile)
        rows = [row for row in spamreader][1:]

    token_transfers = dict()
    for row in rows:
        if row[2] in pools:
            assert row[3] not in pools
            pool = row[2]
            operation = Operation.REMOVAL
        elif row[3] in pools:
            pool = row[3]
            operation = Operation.ADDITION
        else:
            assert False
        if pool not in token_transfers:
            token_transfers[pool] = []
        token_transfers[pool].append(TokenTransfer(int(row[0]), pool, row[1], operation, int(row[4], 16)))
    
    print('Got token transfers for {} pools'.format(len(token_transfers)))
    
    return token_transfers


def get_lp_transfers():
    with open('data/bq-lp.csv') as csvfile:
        spamreader = csv.reader(csvfile)
        rows = [row for row in spamreader][1:]

    lp_transfers = dict()
    for row in rows:
        if row[1] not in lp_transfers:
            lp_transfers[row[1]] = []
        lp_transfers[row[1]].append(LPTransfer(int(row[0]), row[1], row[2], row[3], int(row[4], 16)))
    
    print('Got lp transfers for {} pools'.format(len(lp_transfers)))
    
    return lp_transfers


def get_eth_prices():
    with open('data/eth.csv') as csvfile:
        spamreader = csv.reader(csvfile)
        rows = [row for row in spamreader][1:]
        return [Price(int(float(row[0])), 10 ** 20 // int(float(row[1]) * 100)) for row in rows]

    
def get_usdc_prices():
    return [Price(START_TIMESTAMP, 10 ** 6)]


def get_usdt_prices():
    return [Price(START_TIMESTAMP, 10 ** 6)]


def get_dai_prices():
    return [Price(START_TIMESTAMP, 10 ** 18)]



def calculate_users_usdsec(all_events, tracked_token):
    user_share = dict()
    user_usd_sec = dict()
    pool_total_supply = 0
    pool_total_tokens = 0
    prev_price = 0
    last_processed_sec = 0

    for op in all_events:
        if last_processed_sec < op.timestamp and prev_price > 0:
            # update all users contributions
            total_usd_sec = (op.timestamp - last_processed_sec) * pool_total_tokens * 2 / prev_price
            if total_usd_sec < 0:
                print(op.timestamp, last_processed_sec, pool_total_tokens, prev_price)
                assert False
            for user, share in user_share.items():
                if user not in user_usd_sec:
                    user_usd_sec[user] = 0
                user_usd_sec[user] += total_usd_sec * share / pool_total_supply

        # process next event
        if type(op).__name__ == LPTransfer.__name__:
            if op.addr_from == ZERO_ADDR:
                # mint
                if op.addr_to != op.pool:
                    if op.addr_to not in user_share:
                        user_share[op.addr_to] = 0
                    user_share[op.addr_to] += op.amount
#                     if op.addr_to == '0x52eb4bd04dea22dc589cc973a96d9e1f9e423c12':
#                         print(op, user_share[op.addr_to])
                pool_total_supply += op.amount
            elif op.addr_to == ZERO_ADDR:
                # burn
                if op.addr_from not in user_share:
                    user_share[op.addr_from] = 0
                    assert op.amount == 0
#                     if op.addr_from == '0x52eb4bd04dea22dc589cc973a96d9e1f9e423c12':
#                         print(op, user_share[op.addr_from])
                user_share[op.addr_from] -= op.amount
                pool_total_supply -= op.amount
            else:
                # transfer
                if op.addr_from not in user_share:
                    user_share[op.addr_from] = 0
                    assert op.amount == 0
                if op.addr_to not in user_share:
                    user_share[op.addr_to] = 0
                user_share[op.addr_to] += op.amount
                user_share[op.addr_from] -= op.amount
#                 if op.addr_from == '0x52eb4bd04dea22dc589cc973a96d9e1f9e423c12':
#                     print(op, user_share[op.addr_from])
#                 if op.addr_to == '0x52eb4bd04dea22dc589cc973a96d9e1f9e423c12':
#                     print(op, user_share[op.addr_to])
        elif type(op).__name__ == TokenTransfer.__name__:
            if op.token == tracked_token:
                if op.operation == Operation.ADDITION:
                    pool_total_tokens += op.amount
                elif op.operation == Operation.REMOVAL:
                    pool_total_tokens -= op.amount
                else:
                    assert False
        elif type(op).__name__ == Price.__name__:
            prev_price = op.price
        else:
            assert False
        last_processed_sec = op.timestamp

    return user_usd_sec

In [ ]:
lp_transfers = get_lp_transfers()
pools = list(lp_transfers.keys())
eth_transfers = get_eth_transfers(pools)
token_transfers = get_token_transfers(pools)

all_transfers = {
    pool: sorted(lp_transfers[pool] + token_transfers[pool] + (eth_transfers.get(pool) or []), key=attrgetter('timestamp'))
    for pool in pools
}

usdc_prices = get_usdc_prices()
usdt_prices = get_usdt_prices()
dai_prices = get_dai_prices()
eth_prices = get_eth_prices()

In [ ]:
all_usd_sec = dict()

for pool, transfers in all_transfers.items():
#     if pool != '0x3863fc8c1cc59f160280f5d3e4c1a4c63f945ce3':
#         continue
    tracked_token = None
    prices = None
    for transfer in transfers:
        if type(transfer).__name__ == TokenTransfer.__name__:
            if transfer.token == USDC_ADDR:
                tracked_token = USDC_ADDR
                prices = usdc_prices
            elif transfer.token == USDT_ADDR:
                tracked_token = USDT_ADDR
                prices = usdt_prices
            elif transfer.token == DAI_ADDR:
                tracked_token = DAI_ADDR
                prices = dai_prices
            elif transfer.token == ZERO_ADDR:
                tracked_token = ZERO_ADDR
                prices = eth_prices
            else:
                continue
            break
    if tracked_token is None:
        continue
    events = sorted(transfers + prices, key=attrgetter('timestamp'))
    user_usd_sec = calculate_users_usdsec(events, tracked_token)
    all_usd_sec[pool] = user_usd_sec
#     print(user_usd_sec['0x52eb4bd04dea22dc589cc973a96d9e1f9e423c12'])
    print('processed', pool)

In [ ]:
total_usd_sec = []
user_usd_sec = dict()

for pool_usd_sec in all_usd_sec.values():
    for user, amount in pool_usd_sec.items():
        if user not in user_usd_sec:
            user_usd_sec[user] = []
        user_usd_sec[user].append(amount)
        total_usd_sec.append(amount)

total_usd_sec = int(sum(sorted(total_usd_sec)))

tokens = {
    user: TOTAL_INCH_AIRDROP * int(sum(sorted(usd_sec))) // total_usd_sec
    for user, usd_sec in user_usd_sec.items()
}

print(sum(tokens.values()))
print(TOTAL_INCH_AIRDROP)

with open('users.csv', 'w') as out_f:
    print('user,tokens', file=out_f)
    for user, amount in sorted(tokens.items(), key=itemgetter(1)):
        print('{},{}'.format(user, amount / 1e18), file=out_f)